# LLM Fine-tuning with TRL - Complete Toolkit

Complete toolkit for fine-tuning language models with dataset creation, safeguard management, and advanced features.

In [ ]:
# Install required packages
!pip install torch transformers datasets tokenizers trl peft accelerate bitsandbytes

In [ ]:
import os
import torch
import json
import pandas as pd
import gc
from datasets import Dataset, load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    BitsAndBytesConfig,
    GenerationConfig
)
from trl import SFTTrainer
from peft import LoraConfig, TaskType

# Disable Weights & Biases
os.environ["WANDB_DISABLED"] = "true"

print("All packages imported successfully!")

# Dataset Creation Tools

In [ ]:
class DatasetCreator:
    """Tool for creating training datasets from various sources"""
    
    @staticmethod
    def from_text_file(file_path, chunk_size=512, overlap=50):
        """Create dataset from a plain text file"""
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
        
        chunks = []
        start = 0
        text_length = len(text)
        
        while start < text_length:
            end = start + chunk_size
            chunk = text[start:end]
            chunks.append({"text": chunk})
            start += chunk_size - overlap
        
        return Dataset.from_list(chunks)
    
    @staticmethod
    def from_instruction_pairs(instructions, responses, system_prompt=None):
        """Create dataset from instruction-response pairs"""
        data = []
        for instr, resp in zip(instructions, responses):
            if system_prompt:
                formatted_text = f"### System: {system_prompt}\n### Instruction: {instr}\n### Response: {resp}"
            else:
                formatted_text = f"### Instruction: {instr}\n### Response: {resp}"
            data.append({"text": formatted_text})
        
        return Dataset.from_list(data)
    
    @staticmethod
    def from_jsonl(file_path, text_field="text"):
        """Create dataset from JSONL file"""
        data = []
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                item = json.loads(line)
                data.append({"text": item[text_field]})
        
        return Dataset.from_list(data)
    
    @staticmethod
    def from_csv(file_path, text_column="text"):
        """Create dataset from CSV file"""
        df = pd.read_csv(file_path)
        data = [{"text": str(row[text_column])} for _, row in df.iterrows()]
        return Dataset.from_list(data)
    
    @staticmethod
    def create_conversation_dataset(conversations, system_message=None):
        """Create dataset from conversation history"""
        data = []
        for conv in conversations:
            formatted_conv = ""
            if system_message:
                formatted_conv += f"System: {system_message}\n"
            formatted_conv += "\n".join([f"{role}: {text}" for role, text in conv])
            data.append({"text": formatted_conv})
        
        return Dataset.from_list(data)

dataset_creator = DatasetCreator()
print("Dataset Creator initialized!")

# Safeguard Management & Advanced Tools

In [ ]:
class ModelSafeguardManager:
    """Tools for managing model safeguards and safety features"""
    
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        self.original_generation_config = None
        
    def remove_safety_filters(self):
        """Remove safety filters from the model (use with caution)"""
        print("Removing safety filters...")
        
        self.original_generation_config = self.model.generation_config
        
        permissive_config = GenerationConfig.from_dict({
            "do_sample": True,
            "temperature": 0.8,
            "top_p": 0.95,
            "top_k": 50,
            "max_new_tokens": 512,
            "repetition_penalty": 1.1,
            "pad_token_id": self.tokenizer.eos_token_id,
            "eos_token_id": self.tokenizer.eos_token_id,
        })
        
        self.model.generation_config = permissive_config
        print("Safety filters removed - model is now in permissive mode")
        
    def restore_safety_filters(self):
        """Restore original safety filters"""
        if self.original_generation_config:
            self.model.generation_config = self.original_generation_config
            print("Safety filters restored")
        else:
            print("No original config found to restore")
    
    def set_permissive_generation_config(self, **kwargs):
        """Set custom permissive generation parameters"""
        default_config = {
            "do_sample": True,
            "temperature": 0.9,
            "top_p": 0.95,
            "top_k": 0,
            "max_new_tokens": 1024,
            "repetition_penalty": 1.0,
            "no_repeat_ngram_size": 0,
        }
        default_config.update(kwargs)
        
        permissive_config = GenerationConfig.from_dict(default_config)
        self.model.generation_config = permissive_config
        print("Custom permissive generation config applied")

class ModelEnhancer:
    """Tools for enhancing model capabilities and features"""
    
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
    
    def optimize_for_inference(self):
        """Optimize model for faster inference"""
        print("Optimizing model for inference...")
        self.model.eval()
        if hasattr(self.model, 'config'):
            self.model.config.use_cache = True
        print("Model optimized for inference")

class AdvancedInference:
    """Advanced inference tools with multiple generation strategies"""
    
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
    
    def generate_creative(self, prompt, max_length=200, temperature=0.9):
        """Generate creative text with high temperature"""
        inputs = self.tokenizer.encode(prompt, return_tensors="pt").to(self.model.device)
        
        with torch.no_grad():
            outputs = self.model.generate(
                inputs,
                max_length=max_length,
                temperature=temperature,
                do_sample=True,
                top_p=0.95,
                top_k=50,
                repetition_penalty=1.1,
                pad_token_id=self.tokenizer.eos_token_id
            )
        
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    def generate_deterministic(self, prompt, max_length=200):
        """Generate deterministic text with greedy decoding"""
        inputs = self.tokenizer.encode(prompt, return_tensors="pt").to(self.model.device)
        
        with torch.no_grad():
            outputs = self.model.generate(
                inputs,
                max_length=max_length,
                do_sample=False,
                num_beams=1,
                pad_token_id=self.tokenizer.eos_token_id
            )
        
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    def batch_generate(self, prompts, max_length=100):
        """Generate responses for multiple prompts"""
        inputs = self.tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to(self.model.device)
        
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_length=max_length,
                do_sample=True,
                temperature=0.7,
                pad_token_id=self.tokenizer.eos_token_id
            )
        
        responses = []
        for i, output in enumerate(outputs):
            response = self.tokenizer.decode(output, skip_special_tokens=True)
            response = response[len(prompts[i]):].strip()
            responses.append(response)
        
        return responses

print("Advanced tools classes defined!")

# Create Sample Dataset

In [ ]:
# Create sample text file
sample_text = """Machine learning is a subset of artificial intelligence that focuses on algorithms that can learn from data.
Deep learning uses neural networks with multiple layers to process complex patterns in data.
Natural Language Processing (NLP) enables computers to understand and generate human language.
Transformers are a type of neural network architecture that has revolutionized NLP tasks.
Fine-tuning allows pre-trained models to be adapted to specific tasks with limited data.
Large Language Models (LLMs) are trained on vast amounts of text data and can generate human-like text.
Reinforcement Learning from Human Feedback (RLHF) is used to align models with human preferences."""

with open('/content/sample_data.txt', 'w') as f:
    f.write(sample_text)

print("Sample text file created!")

In [ ]:
# Create instruction dataset
instructions = [
    "Explain machine learning",
    "What is deep learning?",
    "How does NLP work?",
    "What are Transformers in AI?",
    "Explain fine-tuning in simple terms"
]

responses = [
    "Machine learning is a branch of AI that enables computers to learn from data without being explicitly programmed.",
    "Deep learning uses multi-layered neural networks to learn complex patterns from large amounts of data.",
    "NLP, or Natural Language Processing, involves teaching computers to understand, interpret, and generate human language.",
    "Transformers are neural network architectures that use self-attention mechanisms to process sequential data efficiently.",
    "Fine-tuning is like giving a pre-trained AI model additional specialized training for a specific task or domain."
]

instruction_dataset = dataset_creator.from_instruction_pairs(instructions, responses)
print("Instruction dataset created!")
print(f"Number of examples: {len(instruction_dataset)}")
for i, example in enumerate(instruction_dataset):
    print(f"\nExample {i+1}:")
    print(example['text'])

# Model Configuration & Training

In [ ]:
# Configuration
MODEL_NAME = "microsoft/DialoGPT-small"
OUTPUT_DIR = "./results"
MAX_SEQ_LENGTH = 512

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

print("Tokenizer loaded successfully!")

In [ ]:
# Prepare dataset
dataset = Dataset.from_dict({"train": instruction_dataset})
print(f"Dataset loaded: {len(dataset['train'])} examples")

In [ ]:
# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# Load model with quantization
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

print("Model loaded successfully!")

In [ ]:
# LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"]
)

print("LoRA configuration created!")

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=500,
    fp16=True,
    optim="paged_adamw_32bit",
    report_to="none",
    remove_unused_columns=False,
)

print("Training arguments configured!")

In [ ]:
# Initialize SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    tokenizer=tokenizer,
    peft_config=peft_config,
    packing=True,
)

print("Trainer initialized successfully!")

In [ ]:
# Start training
print("Starting training...")
trainer.train()

# Save the trained model
trainer.save_model()
tokenizer.save_pretrained(OUTPUT_DIR)
print("Model saved successfully!")

# Advanced Features & Safeguard Management

In [ ]:
# Initialize advanced tools
safeguard_manager = ModelSafeguardManager(model, tokenizer)
model_enhancer = ModelEnhancer(model, tokenizer)
advanced_inference = AdvancedInference(model, tokenizer)

print("Advanced tools initialized!")

In [ ]:
# Test basic generation
def test_basic_generation():
    test_prompt = "### Instruction: Explain what machine learning is.\n### Response:"
    
    inputs = tokenizer.encode(test_prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=150,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

print("Basic generation test:")
basic_response = test_basic_generation()
print(basic_response)

In [ ]:
# Demonstrate safeguard removal (USE WITH CAUTION)
print("\n=== Safeguard Management Demo ===")

# Remove safety filters
safeguard_manager.remove_safety_filters()

# Set custom permissive generation config
safeguard_manager.set_permissive_generation_config(
    temperature=0.9,
    top_p=0.95,
    max_new_tokens=200,
    repetition_penalty=1.0
)

print("\nModel is now in permissive mode with reduced safeguards")

In [ ]:
# Test advanced generation methods
print("\n=== Advanced Generation Methods ===")

test_prompts = [
    "Explain the concept of artificial intelligence",
    "What are the benefits of machine learning?",
    "How do neural networks work?"
]

# Test creative generation
print("\n1. Creative Generation (High Temperature):")
creative_response = advanced_inference.generate_creative(
    test_prompts[0], 
    max_length=150, 
    temperature=0.9
)
print(creative_response)

# Test batch generation
print("\n2. Batch Generation:")
batch_responses = advanced_inference.batch_generate(test_prompts, max_length=80)
for i, response in enumerate(batch_responses):
    print(f"\nPrompt {i+1}: {test_prompts[i]}")
    print(f"Response: {response}")

In [ ]:
# Model enhancement features
print("\n=== Model Enhancement Features ===")

# Optimize for inference
model_enhancer.optimize_for_inference()

print("\nModel enhancement completed!")

In [ ]:
# Safeguard restoration
print("\n=== Safeguard Restoration ===")

# Restore safety filters
safeguard_manager.restore_safety_filters()

print("Safety features have been restored")

# File Upload & Custom Data

In [ ]:
from google.colab import files

def upload_file():
    """Upload a file to Colab"""
    uploaded = files.upload()
    for filename in uploaded.keys():
        print(f'Uploaded {filename} ({len(uploaded[filename])} bytes)')
    return list(uploaded.keys())[0] if uploaded else None

print("File upload function ready!")
print("To use: uploaded_file = upload_file()")

In [ ]:
# Process uploaded file
def process_uploaded_file(filename):
    """Process uploaded file based on extension"""
    if filename.endswith('.txt'):
        return dataset_creator.from_text_file(filename)
    elif filename.endswith('.jsonl'):
        return dataset_creator.from_jsonl(filename)
    elif filename.endswith('.csv'):
        return dataset_creator.from_csv(filename)
    else:
        raise ValueError(f"Unsupported file type: {filename}")

print("File processing function ready!")
print("To use: custom_dataset = process_uploaded_file(uploaded_file)")

# Dataset Quality Analysis

In [ ]:
class DatasetAnalyzer:
    """Tools for analyzing dataset quality"""
    
    @staticmethod
    def analyze_text_lengths(dataset, text_field="text"):
        """Analyze text length distribution"""
        lengths = [len(text.split()) for text in dataset[text_field]]
        
        print(f"Text Length Analysis:")
        print(f"  Total examples: {len(lengths)}")
        print(f"  Average length: {sum(lengths) / len(lengths):.1f} words")
        print(f"  Min length: {min(lengths)} words")
        print(f"  Max length: {max(lengths)} words")
        
        return lengths
    
    @staticmethod
    def show_samples(dataset, num_samples=3, text_field="text"):
        """Show sample examples from dataset"""
        print(f"\nSample examples (showing {num_samples}):")
        for i in range(min(num_samples, len(dataset))):
            print(f"\n--- Example {i+1} ---")
            print(dataset[i][text_field][:500] + "..." if len(dataset[i][text_field]) > 500 else dataset[i][text_field])

# Analyze current dataset
analyzer = DatasetAnalyzer()
print("\n=== Dataset Analysis ===")
lengths = analyzer.analyze_text_lengths(dataset['train'])
analyzer.show_samples(dataset['train'])

# Model Export Tools

In [ ]:
class ModelExporter:
    """Tools for exporting models to different formats"""
    
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
    
    def create_deployment_package(self, output_dir):
        """Create a deployment package with model and necessary files"""
        import shutil
        
        # Create deployment directory
        deploy_dir = os.path.join(output_dir, "deployment_package")
        os.makedirs(deploy_dir, exist_ok=True)
        
        # Copy model files
        if os.path.exists(OUTPUT_DIR):
            shutil.copytree(OUTPUT_DIR, os.path.join(deploy_dir, "model"), dirs_exist_ok=True)
        
        # Create requirements file
        requirements = """torch>=2.0.0
transformers>=4.35.0
accelerate>=0.24.0
peft>=0.6.0
"""
        
        with open(os.path.join(deploy_dir, "requirements.txt"), "w") as f:
            f.write(requirements)
        
        # Create inference script
        inference_script = """
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

def load_model(model_path):
    model = AutoModelForCausalLM.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    return model, tokenizer

def generate_response(model, tokenizer, prompt, max_length=100):
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(inputs, max_length=max_length, do_sample=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

if __name__ == "__main__":
    model, tokenizer = load_model("./model")
    response = generate_response(model, tokenizer, "Hello, how are you?")
    print(response)
"""
        
        with open(os.path.join(deploy_dir, "inference.py"), "w") as f:
            f.write(inference_script)
        
        print(f"Deployment package created at: {deploy_dir}")

# Initialize exporter
model_exporter = ModelExporter(model, tokenizer)

# Create deployment package
model_exporter.create_deployment_package("./")

print("\nModel export completed!")

# Summary

## 🎯 Complete Toolkit Features

### **Dataset Creation**
- ✅ Text file processing with chunking
- ✅ Instruction-response pairs
- ✅ JSONL/CSV file support
- ✅ Conversation dataset creation
- ✅ File upload integration

### **Model Training**
- ✅ TRL SFTTrainer integration
- ✅ LoRA efficient fine-tuning
- ✅ 4-bit quantization
- ✅ Memory-optimized training

### **Safeguard Management**
- ✅ Safety filter removal/restoration
- ✅ Permissive generation configs
- ✅ Advanced inference modes
- ✅ Batch generation

### **Advanced Features**
- ✅ Creative & deterministic generation
- ✅ Model optimization
- ✅ Quality analysis tools
- ✅ Deployment packaging

### **Safety Notes** ⚠️
- Use safeguard removal responsibly
- Restore filters after testing
- Follow ethical AI guidelines
- Monitor model outputs

The notebook provides a complete pipeline from data creation to model deployment with professional-grade tools!